In [1]:
import os
os.chdir('/Users/azizbenmosbah/PycharmProjects/nlp-getting-started')
import pandas as pd 
from Pipelines.main import tfidf_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
X_train, X_test, y_train, y_test = tfidf_pipeline('/Users/azizbenmosbah/PycharmProjects/nlp-getting-started/Data/train.csv')

In [5]:
X_train

,id,keyword,location,text,00,05,08,10,11,15,...,years,yes,youtube,zone,û_,ûª,ûªs,ûªt,ûò,ûó
0,5187,fatalities,"Youngstown, OH",OSP concerned about mounting fatalities http:/...,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
1,981,blazing,,REAL ViBEZ RADIO - BLAZING THE BEST VIBEZ!!! h...,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,4341,dust%20storm,"Atlanta, GA",@deadlydemi even staying up all night to he ba...,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
3,80,ablaze,South Africa,TRUCK ABLAZE : R21. VOORTREKKER AVE. OUTSIDE O...,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
4,6925,mass%20murderer,"Huntsville, AL",@TheEconomist Step one: get that mass murderer...,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
5,3887,detonation,,Ignition Knock (Detonation) Sensor ACDelco GM ...,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
6,2003,bush%20fires,Selangor,California Bush fires please evacuate affected...,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
7,3542,derailment,"Minneapolis,MN,US",Train derailment: In Patna no news of any casu...,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
8,6318,hostage,,I sent my emails why are the TRINNA hold me ho...,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
9,4322,dust%20storm,SD |Norway| KSA,dust storm in riyadh ????,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0


In [3]:
X_train.head()

,id,keyword,location,text,00,05,08,10,11,15,...,years,yes,youtube,zone,û_,ûª,ûªs,ûªt,ûò,ûó
0,5187,fatalities,"Youngstown, OH",OSP concerned about mounting fatalities http:/...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,981,blazing,,REAL ViBEZ RADIO - BLAZING THE BEST VIBEZ!!! h...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4341,dust%20storm,"Atlanta, GA",@deadlydemi even staying up all night to he ba...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,80,ablaze,South Africa,TRUCK ABLAZE : R21. VOORTREKKER AVE. OUTSIDE O...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6925,mass%20murderer,"Huntsville, AL",@TheEconomist Step one: get that mass murderer...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
param_grid = {'C': [1.132, 1.135, 1.137], 'penalty':['l1', 'l2'], 'solver':['liblinear']}
clf = GridSearchCV(LogisticRegression(), param_grid, cv = 10, scoring = 'f1')
clf.fit(X_train.loc[:,X_train.columns[4:]], y_train)
clf.best_params_

{'C': 1.135, 'penalty': 'l2', 'solver': 'liblinear'}

In [7]:
confusion_matrix(y_test,clf.predict(X_test.loc[:,X_test.columns[4:]]))

array([[759, 102],
       [242, 419]])

In [9]:
print(classification_report(y_test,clf.predict(X_test.loc[:,X_test.columns[4:]]), labels=[1]))

              precision    recall  f1-score   support

           1       0.80      0.63      0.71       661

   micro avg       0.80      0.63      0.71       661
   macro avg       0.80      0.63      0.71       661
weighted avg       0.80      0.63      0.71       661



In [11]:
X_test[clf.predict(X_test.loc[:,X_test.columns[4:]])==1]['text'].values

array(["VIDEO: 'We're picking up bodies from water': Rescuers are searching for hundreds of migrants in the Mediterran... http://t.co/PsPm3ahGKQ",
       'Obama Declares Disaster for Typhoon-Devastated Saipan: Obama signs disaster declaration for Northern Marians a... http://t.co/Q3DtOqO04c',
       "Don't miss Chris #Appy's detonation of the myths obscuring our crime in #Hiroshima 70 yrs ago this month. @salon http://t.co/DlP8kPkt2k",
       'Watch This Airport Get Swallowed Up By A Sandstorm In Under A Minute http://t.co/sEquWmvFx4',
       "Wreckage 'Conclusively Confirmed' as From MH370: Malaysia PM: Investigators and the families of those who were... http://t.co/nn6Y0fD3l0",
       'Madhya Pradesh Train Derailment: Village Youth Saved Many Lives',
       '#USGS M 0.9 - Northern California: Time2015-08-06 01:50:25 UTC2015-08-06 01:50:25 UTC at epicenter... http://t.co/mBo6OAnIQI #EarthTwerk',
       'Live Cricket Score In All Match International\nDomestic\nTeam Tour Team Squad\nPro